In [1]:
# Dependencies and Setup
import os
import csv
import math
import statistics
import pandas as pd

In [2]:
# File to Load
school_data_row = "Resources/schools_complete.csv"
student_data_row = "Resources/students_complete.csv"
print(f"Both files were found")

Both files were found


In [3]:
# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_data_row)

# Read School and Student Data File and store into Pandas Data Frames
student_df = pd.read_csv(student_data_row)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

# See as a DataFrame and check it out
data_df = pd.DataFrame(school_data_complete)
data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
data_df["budget"].sum()

82932329558

In [5]:
data_df["budget"].mean()

2117240.989481746

In [ ]:
data_df.tail()

In [ ]:
data_df.loc[data_df["reading_score"]>=70]

In [ ]:
data_df.loc[data_df["math_score"]>=70]

In [ ]:
data_df.describe()

In [ ]:
data_df["Pass"] = data_df["math_score"]
    

In [ ]:
data_df[loc[data_df["math_score"]>=70]].mean()

In [ ]:
# District Summary
# Create a high level snapshot (in table form) of the district's key metrics, including:
# Total Schools
data_df["school_name"].nunique()

# Total Students
data_df["Student ID"].nunique()

# Total Budget
# Average Math Score
data_df["math_score"].mean()

# Average Reading Score
data_df["reading_score"].mean()

# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)